<a href="https://colab.research.google.com/github/shahzaibrazzaq/Ai-personal-life-manager/blob/main/Document_reader_slm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers accelerate sentence-transformers faiss-cpu gradio pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.0/331.0 kB 29.7 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)

llm = pipeline("text-generation", model=model, tokenizer=tokenizer)
print("✅ Model loaded (cached if already downloaded)")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✅ Model loaded (cached if already downloaded)


In [3]:
from pypdf import PdfReader

def read_document(file):
    text = ""
    if file.name.endswith(".pdf"):
        reader = PdfReader(file.name)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    else:
        text = file.read().decode("utf-8")
    return text


In [4]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embedder = SentenceTransformer("all-MiniLM-L6-v2")

def create_index(text):
    chunks = [text[i:i+500] for i in range(0, len(text), 500)]
    embeddings = embedder.encode(chunks)
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(np.array(embeddings))
    return index, chunks


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
def answer_question(question, index, chunks):
    q_embed = embedder.encode([question])
    D, I = index.search(np.array(q_embed), k=2)

    context = ""
    for i in I[0]:
        context += chunks[i] + "\n"

    prompt = f"""
Use the following document to answer:

{context}

Question: {question}
Answer:
"""

    output = llm(prompt, max_new_tokens=200)[0]["generated_text"]
    return output.split("Answer:")[-1]


In [10]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# ========== MODEL LOAD ==========
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)
llm = pipeline("text-generation", model=model, tokenizer=tokenizer)
embedder = SentenceTransformer("all-MiniLM-L6-v2")

doc_index = None
doc_chunks = None

# ========== FUNCTIONS ==========
def read_document(file):
    text = ""
    if file.name.endswith(".pdf"):
        reader = PdfReader(file.name)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    else:
        text = file.read().decode("utf-8")
    return text

def create_index(text):
    chunks = [text[i:i+500] for i in range(0, len(text), 500)]
    embeddings = embedder.encode(chunks)
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(np.array(embeddings))
    return index, chunks

def upload_file(file):
    global doc_index, doc_chunks
    if file is None: return "❌ Please select a file."
    text = read_document(file)
    doc_index, doc_chunks = create_index(text)
    return "✅ Document loaded! Ask your question below."

def answer_question(question):
    if doc_index is None:
        return "❌ Please upload a document first."

    q_embed = embedder.encode([question])
    D, I = doc_index.search(np.array(q_embed), k=2)

    context = ""
    for i in I[0]:
        context += doc_chunks[i] + "\n"

    prompt = f"<|system|>\nAnswer based on context.</s>\n<|user|>\nContext: {context}\nQuestion: {question}</s>\n<|assistant|>\n"
    output = llm(prompt, max_new_tokens=250)[0]["generated_text"]
    return output.split("<|assistant|>\n")[-1].strip()

# ========== STYLING & UI ==========
custom_css = """
.container { max-width: 1100px; margin: auto; padding-top: 20px; }
.header-text { text-align: center; margin-bottom: 30px; }
.header-text h1 { color: #38bdf8 !important; font-size: 2.8rem !important; }
.header-text p { color: #94a3b8; }
.status-msg { font-weight: bold; }
"""

# Theme definition
my_theme = gr.themes.Soft(
    primary_hue="sky",
    neutral_hue="slate",
    font=[gr.themes.GoogleFont("Inter"), "sans-serif"]
)

with gr.Blocks(css=custom_css, theme=my_theme) as app:

    # Header Section
    with gr.Column(elem_classes="header-text"):
        gr.Markdown("# 📄 Document Reader")
        gr.Markdown("Upload your research papers, contracts, or notes and chat with them instantly.")

    with gr.Row():
        # Left Panel (Input)
        with gr.Column(scale=1):
            with gr.Group():
                gr.Markdown("### 📂 1. Source")
                file_input = gr.File(label="Upload PDF/TXT", file_types=[".pdf", ".txt"])
                upload_btn = gr.Button("Process Document", variant="primary")
                status_output = gr.Text(label="System Log", placeholder="Awaiting upload...", interactive=False)

        # Right Panel (Chat)
        with gr.Column(scale=2):
            with gr.Group():
                gr.Markdown("### 💬 2. Query & Analysis")
                question_input = gr.Textbox(
                    label="Ask a question about the document:",
                    placeholder="What are the key findings?",
                    lines=3
                )
                submit_btn = gr.Button("Search Document", variant="secondary")

                gr.Markdown("---")
                answer_output = gr.TextArea(
                    label="AI Generated Response",
                    placeholder="Insightful answers will appear here...",
                    lines=15, # Expansive output area
                    show_copy_button=True
                )

    # Interactions
    upload_btn.click(upload_file, inputs=file_input, outputs=status_output)
    submit_btn.click(answer_question, inputs=question_input, outputs=answer_output)
    question_input.submit(answer_question, inputs=question_input, outputs=answer_output)

app.launch()

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
/tmp/ipython-input-2178981709.py:79: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=my_theme) as app:
/tmp/ipython-input-2178981709.py:79: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=my_theme) as app:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f5947e70339da05cf9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
